In [24]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [54]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
df_refugee = df_refugee[df_refugee['Year']>1984]
display(df_refugee.shape)
df_refugee = df_refugee.groupby(['Year','Country / territory of asylum/residence','Origin']).sum().reset_index()
display(df_refugee.shape)

1492205

(161831, 5)

(111300, 4)

In [55]:
df_geo  = pd.read_csv("geo.csv",na_values ='NaN',keep_default_na =False)
display(df_geo.head())

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.93911,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [60]:
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['Year','latitude','longitude','country','Value','Origin']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_asylum','longitude': 'longitude_asylum','country':'country_asylum'})
display(df_merge.head(10))

,Year,latitude_asylum,longitude_asylum,country_asylum,Value,Origin
0,1985,28.033886,1.659626,DZ,100.0,Chile
1,1985,28.033886,1.659626,DZ,2000.0,Palestinian
2,1985,28.033886,1.659626,DZ,7060.0,Various/Unknown
3,1985,28.033886,1.659626,DZ,165000.0,Western Sahara
4,1986,28.033886,1.659626,DZ,80.0,Chile
5,1986,28.033886,1.659626,DZ,2000.0,Palestinian
6,1986,28.033886,1.659626,DZ,140.0,Various/Unknown
7,1986,28.033886,1.659626,DZ,165000.0,Western Sahara
8,1987,28.033886,1.659626,DZ,2000.0,Palestinian
9,1987,28.033886,1.659626,DZ,165000.0,Western Sahara


In [61]:
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
display(df_merge.head())
df_merge = df_merge[['Year','country','latitude','longitude','country_asylum','latitude_asylum','longitude_asylum','Value']]
df_merge = df_merge.rename(columns={'latitude': 'latitude_origin','longitude': 'longitude_origin','country':'country_origin'})
df_merge = df_merge.sort_values("Year")
display(df_merge.head())

,Year,latitude_asylum,longitude_asylum,country_asylum,Value,Origin,country,latitude,longitude,name
0,1985,28.033886,1.659626,DZ,100.0,Chile,CL,-35.675147,-71.542969,Chile
1,1986,28.033886,1.659626,DZ,80.0,Chile,CL,-35.675147,-71.542969,Chile
2,1985,-38.416097,-63.616672,AR,80.0,Chile,CL,-35.675147,-71.542969,Chile
3,1986,-38.416097,-63.616672,AR,1690.0,Chile,CL,-35.675147,-71.542969,Chile
4,1993,-38.416097,-63.616672,AR,412.0,Chile,CL,-35.675147,-71.542969,Chile


,Year,country_origin,latitude_origin,longitude_origin,country_asylum,latitude_asylum,longitude_asylum,Value
0,1985,CL,-35.675147,-71.542969,DZ,28.033886,1.659626,100.0
86593,1985,ID,-0.789275,113.921327,PG,-6.314993,143.95555,11820.0
62240,1985,LK,7.873054,80.771797,IT,41.87194,12.56738,10.0
32863,1985,GW,11.803749,-15.180413,IT,41.87194,12.56738,2.0
53127,1985,ZW,-19.015438,29.154857,BW,-22.328474,24.684866,4250.0


In [62]:
display(df_merge.shape)

(93843, 8)

In [63]:
display(df_merge.shape)

(93843, 8)

In [64]:
df_merge.to_csv("final_data.csv", sep=',', encoding='utf-8',index = False)